## MealsCount Config Parser  
  
This notebook contains functionality to parse the MealsCount configuration file (in JSON format). The file is used by the MealsCount backend service to read-in values used to arrive at groupings of schools for near-optimal CEP coverage within a school district.    

In [1]:
import sys
import os
import json
import pandas as pd
import numpy as np

In [2]:
def parseJSON(self,cfgfile):
    
    try:
        with open(cfgfile) as f:
            jsondata = json.load(f)
    except ValueError as ve:
        print("Failed to parse {}".format(cfgfile))
        raise ve
    except Exception as e:
        raise e  
    
    return jsondata

In [3]:
class mcConfig:
    """
    Implementation for MealsCount configuration parser and data store.
    """        
    
    def __init__(self, cfgfile):
        self.__err_status = False
        self.__cfgfile = cfgfile
        try:
            self.__cfgdata = self.__parse(self.__cfgfile)
        except Exception as e:
            self.__err_status = True
            raise e
    
    def status(self):
        return not self.__err_status
    
    def version(self):
        return self.__cfgdata["version"]
    
    def params(self, scope=None):
        if self.status():
            return self.__cfgdata
        else:
            return None
        
    __parse = parseJSON         
        

In [4]:
def displayModelConfig(self,cfgdata):
    
    print("\n")
    print("MealsCount Model Configuration")
    print("------------------------------")    
    print("Version: {}".format(cfgdata["version"]))
    print("Model Variant: {}".format(cfgdata["model_params"]["model_variant"]))
    print("Default ISP Width (%): {}".format(cfgdata["model_params"]["isp_width_default"]))
    print("Min CEP Threshold (%): {}".format(cfgdata["model_params"]["min_cep_thold_pct"]))
    print("Max CEP Threshold (%): {}".format(cfgdata["model_params"]["max_cep_thold_pct"]))
    print("CEP Rates Table:")    
    
    df = pd.DataFrame(cfgdata["model_params"]["cep_rates"])
    df.set_index("region",inplace=True)
    df.index.name=None
    
    print(df)
        

In [5]:
class mcModelConfig(mcConfig):
    """
    Implementation for MealsCount Model Configuration
    """        
    
    def __init__(self, cfgfile):     
        self.__rates_df = None
        self.__regions = None
        self.__cfgfile = cfgfile        
        try:
            mcConfig.__init__(self,cfgfile)
        except Exception as e:
            raise e
    
    def status(self):
        return mcConfig.status(self)
    
    def regions(self):
        if self.__regions is None:
            if self.status():
                self.__regions = mcConfig.params(self)["us_regions"]
                
        return self.__regions
    
    def model_variant(self):
        if self.status():
            return mcConfig.params(self)["model_params"]["model_variant"]
    
    def isp_width(self):
        if self.status():
            return mcConfig.params(self)["model_params"]["isp_width_default"]
        
    def max_cep_thold_pct(self):
        if self.status():
            return mcConfig.params(self)["model_params"]["max_cep_thold_pct"]
        else:
            return -1
    def min_cep_thold_pct(self):
        if self.status():
            return mcConfig.params(self)["model_params"]["min_cep_thold_pct"]
        else:
            return -1
        
    def cep_rates(self,region='default'):
        if self.__rates_df is None:
            if self.status():
                self.__rates_df = pd.DataFrame(mcConfig.params(self)["model_params"]["cep_rates"])
                self.__rates_df.set_index("region",inplace=True)
                self.__rates_df.index.name=None                
                                         
        try:
            cep_rates = self.__rates_df.loc[region]
        except Exception as e:
            # use default rates if no explicit rates found for the region
            # specified (includes both invalid and default regions)            
            cep_rates = self.__rates_df.loc["default"]
                        
        return cep_rates
    
    def show(self):
        if self.status():
            self.__show(mcConfig.params(self))            
        else:
            print("Error: No configuration to display")
        
    def params(self,scope='model'):
        if self.status():
            if scope is "model":
                return mcConfig.params(self)["model_params"]
            else:
                return mcConfig.params(self)
        else:
            return None
    
    __show = displayModelConfig

### Usage  
  
Below code fragments demonstrate the usage of the above functionality.  

In [6]:
CWD = os.getcwd()

CONFIG_FILE = "config.json"

In [7]:
cfg = mcModelConfig(CONFIG_FILE)
cfg.show()



MealsCount Model Configuration
------------------------------
Version: 2.0
Model Variant: v2
Default ISP Width (%): 3.125
Min CEP Threshold (%): 0.4
Max CEP Threshold (%): 0.625
CEP Rates Table:
         nslp_lunch_free_rate  nslp_lunch_paid_rate  sbp_bkfst_free_rate  \
default                  3.23                  0.31                 1.75   
AK                       5.24                  0.50                 2.79   
HI                       3.78                  0.36                 2.03   
PR                       3.78                  0.36                 2.03   

         sbp_bkfst_paid_rate  
default                 0.30  
AK                      0.45  
HI                      0.34  
PR                      0.34  


In [8]:
# True => No Error; False => Error (likely due to errors stemming from parsing config file)
cfg.status()

True

In [9]:
cfg.version()

'2.0'

In [10]:
%pprint
# by default only the model parameters are fetched in raw format (json/Python dict)
cfg.params()

Pretty printing has been turned OFF


{'model_variant': 'v2', 'isp_width_default': 3.125, 'min_cep_thold_pct': 0.4, 'max_cep_thold_pct': 0.625, 'cep_rates': [{'region': 'default', 'nslp_lunch_free_rate': 3.23, 'nslp_lunch_paid_rate': 0.31, 'sbp_bkfst_free_rate': 1.75, 'sbp_bkfst_paid_rate': 0.3}, {'region': 'AK', 'nslp_lunch_free_rate': 5.24, 'nslp_lunch_paid_rate': 0.5, 'sbp_bkfst_free_rate': 2.79, 'sbp_bkfst_paid_rate': 0.45}, {'region': 'HI', 'nslp_lunch_free_rate': 3.78, 'nslp_lunch_paid_rate': 0.36, 'sbp_bkfst_free_rate': 2.03, 'sbp_bkfst_paid_rate': 0.34}, {'region': 'PR', 'nslp_lunch_free_rate': 3.78, 'nslp_lunch_paid_rate': 0.36, 'sbp_bkfst_free_rate': 2.03, 'sbp_bkfst_paid_rate': 0.34}]}

In [11]:
# regions all US states + union territories
cfg.regions()

['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

In [12]:
cfg.model_variant()

'v2'

In [13]:
cfg.isp_width()

3.125

In [14]:
cfg.min_cep_thold_pct()

0.4

In [15]:
cfg.max_cep_thold_pct()

0.625

In [16]:
default_rates = cfg.cep_rates("default")
default_rates

nslp_lunch_free_rate    3.23
nslp_lunch_paid_rate    0.31
sbp_bkfst_free_rate     1.75
sbp_bkfst_paid_rate     0.30
Name: default, dtype: float64

In [17]:
type(default_rates)

<class 'pandas.core.series.Series'>

In [18]:
default_rates["sbp_bkfst_free_rate"]

1.75

In [19]:
# rates for a default region
cfg.cep_rates("CA")

nslp_lunch_free_rate    3.23
nslp_lunch_paid_rate    0.31
sbp_bkfst_free_rate     1.75
sbp_bkfst_paid_rate     0.30
Name: default, dtype: float64

In [20]:
# rates for a non-defualt region
cfg.cep_rates("PR")

nslp_lunch_free_rate    3.78
nslp_lunch_paid_rate    0.36
sbp_bkfst_free_rate     2.03
sbp_bkfst_paid_rate     0.34
Name: PR, dtype: float64

In [21]:
# rates for non-existent/invalid region results in default rates being fetched
cfg.cep_rates("XYZ")

nslp_lunch_free_rate    3.23
nslp_lunch_paid_rate    0.31
sbp_bkfst_free_rate     1.75
sbp_bkfst_paid_rate     0.30
Name: default, dtype: float64

In [22]:
# fetch all parameters by specifying 'all'
params = cfg.params('all')
print(json.dumps(params, indent=2))

{
  "version": "2.0",
  "us_regions": [
    "AL",
    "AK",
    "AZ",
    "AR",
    "CA",
    "CO",
    "CT",
    "DC",
    "DE",
    "FL",
    "GA",
    "HI",
    "ID",
    "IL",
    "IN",
    "IA",
    "KS",
    "KY",
    "LA",
    "ME",
    "MD",
    "MA",
    "MI",
    "MN",
    "MS",
    "MO",
    "MT",
    "NE",
    "NV",
    "NH",
    "NJ",
    "NM",
    "NY",
    "NC",
    "ND",
    "OH",
    "OK",
    "OR",
    "PA",
    "PR",
    "RI",
    "SC",
    "SD",
    "TN",
    "TX",
    "UT",
    "VT",
    "VA",
    "WA",
    "WV",
    "WI",
    "WY"
  ],
  "model_params": {
    "model_variant": "v2",
    "isp_width_default": 3.125,
    "min_cep_thold_pct": 0.4,
    "max_cep_thold_pct": 0.625,
    "cep_rates": [
      {
        "region": "default",
        "nslp_lunch_free_rate": 3.23,
        "nslp_lunch_paid_rate": 0.31,
        "sbp_bkfst_free_rate": 1.75,
        "sbp_bkfst_paid_rate": 0.3
      },
      {
        "region": "AK",
        "nslp_lunch_free_rate": 5.24,
        "nsl

## TODO  
  
* Test for error conditions (malformed files resulting in parsing errors)  
* Add documentation to code (and elsewhere)